# 프로젝트 설계를 위한 MovieLens 데이터셋 구조 확인

In [1]:
import pandas as pd

**userId**  

- MovieLens 사용자는 무작위로 선택되었음. 
- ID는 익명 
- ratings.csv `userId`는 tags.csv의 `userId`와 같다. 


**movieId**  

- 등급이나 태그가 하나 이상 있는 영화만 데이터 세트에 포함
- `movieId`는 MovieLens 웹 사이트에서 사용되는 것과 동일함(예: ID는 https://movielens.org/movies/1 에 해당함). 
- ratings.csv의 `movieId`는 tags.csv, movies.csv, links.csv와 동일한 id

## movies.csv

In [ ]:
df = pd.read_csv("dataset/movies.csv")
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
df.tail()

,movieId,title,genres
86532,288967,State of Siege: Temple Attack (2021),Action|Drama
86533,288971,Ouija Japan (2021),Action|Horror
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),Documentary
86535,288977,Skinford: Death Sentence (2023),Crime|Thriller
86536,288983,UNZIPPED: An Autopsy of American Inequality (2...,Documentary


In [ ]:
len(df)

86537

In [ ]:
genres_list = df["genres"].map(lambda x: x.split("|"))
genres = set()

for l in genres_list:
    genres.update(l)

print(len(genres))
print(genres)

20
{'Adventure', 'War', '(no genres listed)', 'Action', 'Mystery', 'Fantasy', 'Film-Noir', 'Comedy', 'Romance', 'IMAX', 'Crime', 'Musical', 'Animation', 'Drama', 'Horror', 'Children', 'Sci-Fi', 'Thriller', 'Western', 'Documentary'}


In [ ]:
df[df["genres"] == "(no genres listed)"]

,movieId,title,genres
15884,83773,Away with Words (San tiao ren) (1999),(no genres listed)
16063,84768,Glitterbug (1994),(no genres listed)
16354,86493,"Age of the Earth, The (A Idade da Terra) (1980)",(no genres listed)
16494,87061,Trails (Veredas) (1978),(no genres listed)
17410,91246,Milky Way (Tejút) (2007),(no genres listed)
...,...,...,...
86460,288739,Commercial Entertainment Product (1992),(no genres listed)
86464,288749,In Good Conscience: Sister Jeannine Gramick's ...,(no genres listed)
86490,288843,Mongoloid (1978),(no genres listed)
86493,288849,Colaholic (2018),(no genres listed)


In [ ]:
df[df["genres"] == "IMAX"]

,movieId,title,genres
4356,4460,Encounter in the Third Dimension (1999),IMAX


movies.csv는 총 86,537건이고, 데이터 설명대로 `title` 컬럼은 괄호 안에 개봉년도가 포함되어 있습니다.  

[공식 데이터 설명](https://files.grouplens.org/datasets/movielens/ml-latest-README.html)에서는 `genres`에 `IMAX`가 없었는데 업데이트가 안되었나보네요.  

눈에 띄는점은 `genres` 컬럼인데, 각 장르가 `|` 문자로 구분되어 여러개 항목이 들어있습니다. 장르들이 영어 오름차순으로 정렬되어 저장되어있는 것으로 보이네요.  

이러한 경우 일반적인 RDBMS에서 `genres`를 조건으로 이용하여 `SELECT`하게 될 경우 `%{keyword}%` 형식으로 처리해야하므로 인덱스를 활용할 수 없기 때문에 성능에서 문제가 발생할 수 있습니다.  

`genres` 컬럼들의 값을 별도 테이블로 분리하고 중간 테이블을 둔 N:M 구조로 바꾼다면 인덱스를 사용할 수 없는 문제는 해결할 수 있습니다만...  

서비스 요구사항으로 검색 결과 응답에 장르도 포함되어야 한다면, 별도로 장르를 조회하는 처리를 추가한다거나, 테이블 조인이나 서브 쿼리가 필요하게 되므로 항상 성능이 좋다고는 확신할 수 없습니다.  

RDBMS 이외에도 몇가지 방법이 떠오르지만, 일단 RDBMS를 이용한 `genres` 컬럼 조건 검색은 배제하고 이후 더 좋은 방법을 고려하는 것이 좋겠습니다.  

## tags.csv

In [ ]:
df = pd.read_csv("dataset/tags.csv")
df.head()

,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746


In [ ]:
len(df)

2328315

In [ ]:
df_drop = df.drop("timestamp", axis=1).drop_duplicates()
len(df_drop)

2328315

In [ ]:
df_drop = df["tag"].drop_duplicates()
len(df_drop)

153950

`tags.csv`는 사용자가 한 영화에 적용한 태그로 1명의 유저가 한 영화에 여러 종류의 태그를 적용할 수 있습니다.  

`(userId, movieId, tag)`로 묶어 유니크한 것을 확인할 수 있네요.

전체 컬럼수는 2,328,315개 이지만, `tag` 컬럼의 유니크한 값의 개수는 153,950개 인 것을 확인할 수 있습니다.  

`tag`컬럼 값의 중복이 많지만 당장 태그를 이용한 검색을 구현할 예정은 아니고 인덱스 설정만 잘 해준다면 화면에 노출 될 데이터 조회 성능에는 큰 이슈는 없어 보입니다.  


## ratings.csv

In [2]:
df = pd.read_csv("dataset/ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [3]:
len(df)

33832162

In [4]:
print(min(df["rating"]), max(df["rating"]))

0.5 5.0


`ratings.csv`는 영화에 대한 사용자의 평점으로 영화 하나에 하나만 만들 수 있습니다.  

영화 목록을 검색할 때 일반적으로 평균 평점이 포함되는데, 현재 상태로 테이블을 생성할 시 조회 처리에서 `GROUP BY`나 서브 쿼리를 통해 평균을 계산한다던가, 활용된 값들을 조회하는 처리가 필요하게 되고, 이 때문에 쿼리 튜닝이 어려워 지는데요  

`movies` 테이블에 통계값에 활용될 컬럼을 만들어 두는 방식으로 설계하는 것도 고려해볼 수 있겠습니다.  

`rating` 컬럼도 꼭 소수값을 넣을 필요는 없어보이네요.  

## links.csv

- movieId는 https://movielens.org 에서 사용되는 영화의 식별자 
  - 토이 스토리에는 https://movielens.org/movies/1

- imdbId는 http://www.imdb.com 에서 사용되는 영화의 식별자
  - 토이 스토리에는 http://www.imdb.com/title/tt0114709/

- tmdbId는 https://www.themoviedb.org 에서 사용되는 영화의 식별자
  - 토이 스토리에는 https://www.themoviedb.org/movie/862

In [4]:
df = pd.read_csv("dataset/links.csv")
df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


`links.csv`는 굳이 사용하지 않아도 괜찮을 것 같습니다.